In [1]:
%load_ext autoreload
%autoreload 2
import os
from df2d.dataset import Drosophila2Dataset
from df2d.model import Drosophila2DPose
from df2d.parser import create_parser
from df2d.util import pwd
from df2d.inference import path2inp, pr2inp, parse_img_path

import torch
from torch.utils.data import DataLoader
import numpy as np

import matplotlib.pyplot as plt

class load_f:
    def __init__(self, cam_order):
        self.cam_order = cam_order.tolist()
    def __call__(self, x):
        img = plt.imread(x)
        cam_id, _ = parse_img_path(x)
        if self.cam_order.index(cam_id) > 3:
            img = img[:, ::-1]
        return img

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#checkpoint_path = os.path.join(pwd(), "../weights/sh8_deepfly.tar")
checkpoint_path = None
args_default = create_parser().parse_args("").__dict__
model = Drosophila2DPose(checkpoint_path=checkpoint_path, **args_default).to(device)

inp = pr2inp('/home/user/Desktop/DeepFly3D/sample/test/df3d/df3d_result__home_user_Desktop_DeepFly3D_sample_test.pkl')
dat = DataLoader(Drosophila2Dataset(inp, return_heatmap=True, load_f=load_f(np.array([0,1,2,3,4,5,6]))), batch_size=8, shuffle=True)

In [2]:
from pytorch_lightning import Trainer

trainer = Trainer(gpus=1)
trainer.fit(model, dat)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
/home/user/anaconda3/envs/dd2/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type         | Params
---------------------------------------
0 | model | HourglassNet | 6.7 M 
---------------------------------------
6.7 M     Trainable params
0         Non-trainable params
6.7 M     Total params
26.933    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/user/anaconda3/envs/dd2/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

/home/user/anaconda3/envs/dd2/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1